In [54]:
from dspy.datasets import HotPotQA
import dspy
dataset = HotPotQA(train_seed=1, train_size=1000, eval_seed=0, test_size=0)


/home/ec2-user/miniconda3/envs/dspy/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [55]:
# Tell DSPy that the `question` field is the input. Any other fields are labels.
dataset = [x.with_inputs('question') for x in dataset.train]
print(len(dataset))

1000


In [56]:
print(dataset[0])

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})


In [57]:
from dspy.retrieve.chromadb_rm import ChromadbRM
import chromadb
from chromadb.utils import embedding_functions


chroma_client = chromadb.PersistentClient(path='./hotpotqa')
default_ef = embedding_functions.DefaultEmbeddingFunction()

docs = [x.question + " -> " + x.answer for x in dataset]
ids = [f"id{i}" for i in range(len(docs))]
collection = chroma_client.get_or_create_collection(name='hotpotqa', embedding_function=default_ef)

In [58]:
collection.add(
    documents=docs,
    ids=ids
)

Insert of existing embedding ID: id0
Insert of existing embedding ID: id1
Insert of existing embedding ID: id2
Insert of existing embedding ID: id3
Insert of existing embedding ID: id4
Insert of existing embedding ID: id5
Insert of existing embedding ID: id6
Insert of existing embedding ID: id7
Insert of existing embedding ID: id8
Insert of existing embedding ID: id9
Insert of existing embedding ID: id10
Insert of existing embedding ID: id11
Insert of existing embedding ID: id12
Insert of existing embedding ID: id13
Insert of existing embedding ID: id14
Insert of existing embedding ID: id15
Insert of existing embedding ID: id16
Insert of existing embedding ID: id17
Insert of existing embedding ID: id18
Insert of existing embedding ID: id19
Insert of existing embedding ID: id20
Insert of existing embedding ID: id21
Insert of existing embedding ID: id22
Insert of existing embedding ID: id23
Insert of existing embedding ID: id24
Insert of existing embedding ID: id25
Insert of existing emb

In [70]:
retrieval_model = ChromadbRM(
    collection_name='hotpotqa',
    persist_directory='./hotpotqa',
    embedding_function=default_ef,
    k=3,
    client=chroma_client
)

In [71]:
print(retrieval_model("Out of the woords"))

[{'id': 'id860', 'score': 1.3779386281967163, 'long_text': '"Heffalumps and Woozles" is a song from a full-length animated film distributed by who? -> Buena Vista Distribution', 'metadatas': None}, {'id': 'id80', 'score': 1.5018947124481201, 'long_text': 'Hargus Melvin "Pig" Robbins who has played with an American alternative rock band whose highest charting single was what? -> Push th\' Little Daisies', 'metadatas': None}, {'id': 'id922', 'score': 1.5085755586624146, 'long_text': 'Which film out of the following two came out earlier, The Million Dollar Duck or Babes in Toyland? -> Babes in Toyland', 'metadatas': None}]


In [72]:
ollama_model = dspy.OllamaLocal(
    model='phi3',
    model_type='text',
    max_tokens=350,
    temperature=0.7,
    top_p=0.9,
    frequency_penalty=1.17,
    top_k=40
)

In [73]:
ollama_model("Tell me about the weather on jupiter?")

[" Jupiter, being a gas giant, doesn't have a solid surface like Earth. Its atmosphere is composed mainly of hydrogen and helium with traces of other compounds such as methane, water vapor, ammonia crystals and rocky particles. The weather on Jupiter is quite extreme compared to what we experience here on Earth!\n\nThe planet's upper layers boast some fascinating meteorological phenomena:\n1. Great Red Spot (GRS): A giant storm larger than the size of Earth, spinning at high speeds in one of its southern hemispheres for over 300 years. The exact cause and longevity remain subjects to scientific research, but it's thought that Jupiter's rapid rotation contributes significantly to this enduring feature.\n\n2. Zones and Belts: Jupiter has alternating light-colored (zones) and dark belts in its atmosphere. The zones are characterized by high pressure with descending air, while the belts have lower pressures where rising warm gas occurs. These bands of clouds move at different speeds around

In [74]:
# Get Top passages

In [75]:
dspy.settings.configure(rm=retrieval_model, lm=ollama_model)

In [66]:
dev_example = dataset[100]
dev_example

Example({'question': "What was a previous unoffical name for the high performance variant of Audi's compact executive car?", 'answer': 'Audi Ur-S4'}) (input_keys={'question'})

In [76]:
def get_top_passages(question):
    retrieve = dspy.Retrieve(k=3)
    topK_passages = retrieve(question, 3).passages
    print(f"Top {retrieve.k} paggaes for question : {question} \n", '-'*30, '\n')
    for idx, passage in enumerate(topK_passages):
        print(f"{idx+1}]", passage, '\n')
        

In [77]:
get_top_passages(dev_example.question)

Top 3 paggaes for question : What was a previous unoffical name for the high performance variant of Audi's compact executive car? 
 ------------------------------ 

1] What was a previous unoffical name for the high performance variant of Audi's compact executive car? -> Audi Ur-S4 

2] What engine designed to power medium duty trucks and heavy cars was also used in Chevy's intermediate and pony car? -> The Chevrolet "Big Block" 

3] Until 2006, the Nissan Sentra was a rebadged export version of a Japanese car that was first built in what year? -> 1966 



In [78]:
# Define Signatures for Input and Output

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoir answers."""
    context = dspy.InputField()